In [28]:
import re
from __future__ import division
import string
import math
import os.path

with open('/Users/bryanjordan/Sites/server_upload/total_text/total_text_result.txt','r') as f:
    lines = f.read().split("\n")

word = '__BREAK__' 

breakLines = []
hashes = []

for i,line in enumerate(lines):
    if word in line: 
        breakLines.append(i+1)

for hashId in breakLines:
    hashes.append(lines[hashId])

file = open('/Users/bryanjordan/Sites/server_upload/total_text/total_text_result.txt','r') 
 
holder = file.read()

text = {}
listOfHits = []

for match in re.finditer("__BREAK__", holder):
    result = match.start(), match.end()
    listOfHits.append(result)

for entry in listOfHits:
    if listOfHits.index(entry) == 0:
        secondHit = listOfHits[1][0]
        temp = holder[entry[1]:secondHit]
        title = 'text_' + str(0)
        temp = temp.replace("\r"," ")
        temp = temp.replace("\n"," ")
        text[title] = temp
    else:
        
        entryIndex = listOfHits.index(entry)
        originalIndex = entryIndex
        originalIndex += 1
        if originalIndex < len(listOfHits):
            first_index = listOfHits[entryIndex]
            nextIndex = listOfHits[originalIndex]

            temp = holder[first_index[1]:nextIndex[0]]
            title = 'text_' + str(entryIndex)
            temp = temp.replace("\r"," ")
            temp = temp.replace("\n"," ")
            text[title] = temp

        if originalIndex == len(listOfHits):
            first_index = listOfHits[len(listOfHits)-1]

            result = holder[first_index[1]:]
            title = 'text_' + str(len(listOfHits)-1)
            result = result.replace("\r"," ")
            result = result.replace("\n"," ")
            text[title] = result
            
        if originalIndex == len(listOfHits)-1:
            originalIndex = len(listOfHits)-1
            
            first_index = listOfHits[entryIndex]
            nextIndex = listOfHits[originalIndex]

            temp = holder[first_index[1]:nextIndex[0]]
            title = 'text_' + str(entryIndex)
            temp = temp.replace("\r"," ")
            temp = temp.replace("\n"," ")
            text[title] = temp
            
total = []

for entry in text:
    total.append(text[entry])
    

keyNames = []

for key,value in text.items():
    keyNames.append(key)

file.close() 

tokenize = lambda doc: doc.lower().split(" ")

all_documents = total

def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

def sublinear_term_frequency(term, tokenized_document):
    count = tokenized_document.count(term)
    if count == 0:
        return 0
    return 1 + math.log(count)

def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
                
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
            
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(all_documents)

def cosine_similarity(vector1, vector2):
    dot_product = sum(p*q for p,q in zip(vector1, vector2))
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    return dot_product/magnitude

skl_tfidf_comparisons = []
for count_0, doc_0 in enumerate(sklearn_representation.toarray()):
    for count_1, doc_1 in enumerate(sklearn_representation.toarray()):
        first = hashes[count_0]
        second = hashes[count_1]
        skl_tfidf_comparisons.append((cosine_similarity(doc_0, doc_1), first, second))

    
final = sorted(skl_tfidf_comparisons, reverse = True)

def saveToServer(file, titles):
    
    fileStr = ' '.join([str(i) for i in file])
    titlesStr = ' '.join([str(i) for i in titles])
    
    save_path = '/Users/bryanjordan/Sites/server_upload/total_tf_idf/'

    name_of_file = "total_tf_idf_results"

    completeName = os.path.join(save_path, name_of_file+".txt")         

    file1 = open(completeName, "w+")
    
    toWrite = titlesStr + "__CONTENT__" + fileStr

    file1.write(toWrite)

    file1.close()
    print(toWrite)
    print("Complete")
    

saveToServer(final, hashes)

    
#1.if first-key is not in header, add, then go-to 3.
#2. elif first-key is in header go-to 3
#3 if second value not in column, add then go-to 4.
#4 elif second value is in column, add to column


#ONCE THIS IS EXECUTED -> UPLOAD to file -> keep column, parse line by line. reading between values.

#Thursday: (1) upload, (2) compare, (3) statistical analysis?, (4) track user, (5) FAQ search_results spectrum

58867dba526843ce9b3aae69a4f1d1e3 b806ef95d125872d1e72133d96464938 1311f004aa1a268e4309afbb26dcaf1b 067980ba0b78d4f6a134fb324d2e5b23 adac9bba354ed200c0549665c3801d4e 6f58344db84efc46ba42efac4a5de557__CONTENT__(1.0000000000000002, 'b806ef95d125872d1e72133d96464938', 'b806ef95d125872d1e72133d96464938') (1.0000000000000002, 'adac9bba354ed200c0549665c3801d4e', 'adac9bba354ed200c0549665c3801d4e') (1.0000000000000002, '6f58344db84efc46ba42efac4a5de557', '6f58344db84efc46ba42efac4a5de557') (1.0000000000000002, '1311f004aa1a268e4309afbb26dcaf1b', '1311f004aa1a268e4309afbb26dcaf1b') (1.0, '58867dba526843ce9b3aae69a4f1d1e3', '58867dba526843ce9b3aae69a4f1d1e3') (1.0, '067980ba0b78d4f6a134fb324d2e5b23', '067980ba0b78d4f6a134fb324d2e5b23') (0.25326515503835467, 'b806ef95d125872d1e72133d96464938', '067980ba0b78d4f6a134fb324d2e5b23') (0.25326515503835467, '067980ba0b78d4f6a134fb324d2e5b23', 'b806ef95d125872d1e72133d96464938') (0.054486497208124214, '58867dba526843ce9b3aae69a4f1d1e3', '1311f004aa1a268e